# Joining and Appending DataFrames in PySpark HW

Now it's time to test your knowledge and further engrain the concepts we touched on in the lectures. Let's go ahead and get started.




**As always let's start our Spark instance.**

In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("join").getOrCreate()

cores = spark._jsc.sc().getExecutorMemoryStatus().keySet().size()

print(cores)
spark

1


## Read in the database

Let's cotinue working with our college courses dataframe to get some more insights and practice what we have learned! Let's read in the whole database using the loop function that we learned about in the lecture to automatically read in all the datasets from the uw-madision-courses folder (there are too many datasets to each one individually.

In [2]:
import os
path = '../data/uw-madison-courses/'

df_list =[]
for filename in os.listdir(path):
    if filename.endswith(".csv"):
        splitter = filename.split(".")
        df_name = splitter[0]
        df = spark.read.csv(path+filename,inferSchema=True,header=True)
        df_list.append(df_name)
        exec(df_name +("= df"))

print("List of Dataframe that has been extracted...")
print(df_list)


List of Dataframe that has been extracted...
['courses', 'course_offerings', 'grade_distributions', 'instructors', 'rooms', 'schedules', 'sections', 'subjects', 'subject_memberships', 'teachings']


Now check the contents of a few of the dataframses that were read in above.

In [3]:
courses.limit(5).toPandas()

,uuid,name,number
0,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,Cooperative Education Program,1
1,c070a84f-648f-351d-9499-5d0e30ad02cc,Cooperative Education/Co-op in Life Sciences C...,1
2,e6b4b7ae-0e0b-3aa5-9d77-7fcd90c9cfa3,Cooperative Education Program,1
3,8f63bde1-ff7f-3fe7-9901-862908bf134c,Workshop in Dance Activity,1
4,f3541888-584a-3923-9ce7-6341ff3d84a1,Cooperative Education/Co-op in Agricultural & ...,1


## Recap: About this database

You will notice that there are several more tables in the uw-madision-courses folder than there are read in above. This so that you will have a chance to practice your own custom joins and learn about the relationships between a real database work. Sometimes we don't know how they are related and we need to figure it out! I'll save that for the HW :) 

Here is a look at some of the important variables we can use to join our tables:

 - course_offerings: uuid, course_uuid, term_code, name
 - instructors: id, name
 - schedules: uuid
 - sections: uuid, course_offering_uuid,room_uuid, schedule_uuid
 - teachings: instructor_id, section_uuid
 - courses: uuid
 - grade_distributions: course_offering_uuid,section_number
 - rooms: uuid, facility_code, room_code
 - subjects: code
 - subject_memberships: subject_code, course_offering_uuid
 
 **Source:** https://www.kaggle.com/Madgrades/uw-madison-courses
 
So alright, let's use this information to discover some insights from this data!

## 1a. Can you assign the room numbers to each section of each course?

Show only the **room's uuid, facility code, room number, term code and the name of the course** from the **course_offerings table.**

In [4]:
course_offerings.columns

['uuid', 'course_uuid', 'term_code', 'name']

In [5]:
step1= course_offerings.join(sections,course_offerings.uuid == sections.course_offering_uuid)\
        .select(course_offerings.name,course_offerings.term_code,sections.room_uuid).filter(sections.room_uuid != 'null')

step1.limit(5).toPandas()


,name,term_code,room_uuid
0,Cooperative Education Prog,1172,04368a56-c959-3e4b-8b3d-f4cc3538fea5
1,Cooperative Education Prog,1172,04368a56-c959-3e4b-8b3d-f4cc3538fea5
2,Cooperative Education Prog,1114,04368a56-c959-3e4b-8b3d-f4cc3538fea5
3,Cooperative Education Prog,1104,04368a56-c959-3e4b-8b3d-f4cc3538fea5
4,Cooperative Education Prog,1134,04368a56-c959-3e4b-8b3d-f4cc3538fea5


In [6]:
step2 = step1.join(rooms,step1.room_uuid==rooms.uuid)\
        .select(rooms.uuid, rooms.facility_code, rooms.room_code, step1.term_code, step1.name).filter(rooms.room_code !='null')

In [7]:
step2.limit(5).toPandas()

,uuid,facility_code,room_code,term_code,name
0,2cc50da3-ef0e-3572-a557-ca44930a0688,0032,0249,1174,Workshop in Dance Activity
1,ebbf62b4-2ac3-356b-b0fa-7897f4446a17,0032,B101,1174,Workshop in Dance Activity
2,ed828265-475b-31b4-b9a8-daec2a600449,0032,0549,1174,Workshop in Dance Activity
3,ebbf62b4-2ac3-356b-b0fa-7897f4446a17,0032,B101,1174,Workshop in Dance Activity
4,ebbf62b4-2ac3-356b-b0fa-7897f4446a17,0032,B101,1074,Workshop in Dance Activity


## 1b. Now show same output as above but for only facility number 0469 (facility_code)

In [8]:
step2.filter(step2.facility_code== '0469').toPandas()

,uuid,facility_code,room_code,term_code,name
0,9759cb5f-a7d3-3d0c-b8b3-03910bd0153e,0469,2441,1152,Fundamentals-Flute
1,9759cb5f-a7d3-3d0c-b8b3-03910bd0153e,0469,2441,1092,Fundamentals-Flute
2,6af80b0b-b3e3-370a-925f-829869f007bc,0469,4411,1172,Fundamentals-Flute
3,9759cb5f-a7d3-3d0c-b8b3-03910bd0153e,0469,2441,1162,Fundamentals-Flute
4,9759cb5f-a7d3-3d0c-b8b3-03910bd0153e,0469,2441,1132,Fundamentals-Flute
...,...,...,...,...,...
16328,00593bf2-b562-3482-9bb3-e2b5e7e00a16,0469,1101,1182,Sex and Power
16329,0fabcc36-e3c1-32c3-99a9-c7c6ae53e992,0469,1221,1182,Chaucers Courtly Poetry
16330,f4781736-d27f-3902-9095-5b856acb6db3,0469,2340,1182,Graduate Choral Union
16331,f4781736-d27f-3902-9095-5b856acb6db3,0469,2340,1182,Graduate Choral Union


## 2. Count how many sections are offered for each subject for each facility

*Note: this will involve a groupby*

In [24]:
#based off of facility code, enumerate 
step1 = subjects.join(subject_memberships,subjects.code == subject_memberships.subject_code,how="inner")\
        .select(['name','course_offering_uuid']).withColumnRenamed('name','subject_name')

step1.limit(4).toPandas()

,subject_name,course_offering_uuid
0,Chemical and Biological Engineering,344b3ebe-da7e-314c-83ed-9425269695fd
1,Electrical and Computer Engineering,344b3ebe-da7e-314c-83ed-9425269695fd
2,Engineering Mechanics and Astronautics,344b3ebe-da7e-314c-83ed-9425269695fd
3,Mechanical Engineering,344b3ebe-da7e-314c-83ed-9425269695fd


In [25]:
step2 = step1.join(sections,step1.course_offering_uuid==sections.course_offering_uuid,how="left")\
        .select(["subject_name","room_uuid"])

step2.limit(4).toPandas()

,subject_name,room_uuid
0,Communication Arts,dd6118e0-7221-3b81-9b29-aad61f0ede54
1,Communication Arts,4e9c55b8-7c02-36c3-b907-3e832ea285e7
2,Communication Arts,4948c250-c6cf-3272-a497-f90962f3ba67
3,Communication Arts,698555af-8ec3-3b85-8e67-bd7fc53aba80


In [26]:
#I added a filter to make this a little simpler
step3 = step2.join(rooms,step2.room_uuid==rooms.uuid,how="left").filter("facility_code IN('0140','0545','0469','0031')").select(["subject_name","facility_code","room_code"])

step3.limit(4).toPandas()

,subject_name,facility_code,room_code
0,Communication Arts,0140,1070
1,Communication Arts,0545,4028
2,Communication Arts,0545,4008
3,Communication Arts,0469,2241


In [27]:
#option1 :group by facility code and do a count
step3.groupBy("facility_code","subject_name").count().orderBy(["facility_code","count"],descending=["facility_code","count"]).show(10,False)


+-------------+------------------------------------------------+-----+
|facility_code|subject_name                                    |count|
+-------------+------------------------------------------------+-----+
|0031         |Horticulture                                    |1    |
|0031         |Educational Psychology                          |1    |
|0031         |Neurology                                       |1    |
|0031         |History                                         |1    |
|0031         |Interdisciplinary Courses (Engineering)         |1    |
|0031         |Philosophy                                      |1    |
|0031         |La Follette School of Public Affairs            |1    |
|0031         |Environmental Studies - Gaylord Nelson Institute|1    |
|0031         |Interdisciplinary Courses (L&S)                 |1    |
|0031         |History of Science                              |1    |
+-------------+------------------------------------------------+-----+
only s

In [50]:
#option2: group by subejct name and pivot the facility code
#to see each facility side by side within each subject

step3.groupBy("subject_name").pivot("facility_code").count().show(10,False)

+---------------------------------+----+----+----+----+
|subject_name                     |0031|0140|0469|0545|
+---------------------------------+----+----+----+----+
|Asian American Studies           |null|6   |245 |24  |
|Religious Studies                |null|32  |282 |43  |
|HEBREW                           |null|7   |4   |null|
|Botany                           |null|4   |24  |2   |
|Urban and Regional Planning      |null|208 |17  |6   |
|Nutritional Sciences             |6   |null|null|28  |
|Kinesiology                      |1549|null|null|null|
|Art Education (Department of Art)|null|null|145 |null|
|Philosophy                       |1   |80  |1296|73  |
|Microbiology                     |null|null|1   |null|
+---------------------------------+----+----+----+----+
only showing top 10 rows



In [30]:
step3.createOrReplaceTempView("subject")

In [31]:
step3.columns

['subject_name', 'facility_code', 'room_code']

In [71]:
spark.sql("""SELECT  * FROM (SELECT subject_name,facility_code FROM subject group by SUBJECT_NAME,facility_code ) as in_line 
          
          PIVOT(
          COUNT(FACILITY_CODE) as count 
          FOR FACILITY_CODE in ('0031','0140','0469','0545')
          )
          
            """).toPandas()




,subject_name,0031,0140,0469,0545
0,Asian American Studies,NaN,1.0,1.0,1.0
1,Religious Studies,NaN,1.0,1.0,1.0
2,HEBREW,NaN,1.0,1.0,NaN
3,Botany,NaN,1.0,1.0,1.0
4,Urban and Regional Planning,NaN,1.0,1.0,1.0
...,...,...,...,...,...
123,Electrical and Computer Engineering,1.0,1.0,1.0,NaN
124,Zoology,NaN,1.0,1.0,1.0
125,Social Work,NaN,1.0,1.0,NaN
126,Jewish Studies,NaN,1.0,1.0,1.0


## 3. What are the hardest classes?

- Let's see if we can figure out which classes are the hardest by seeing **how many students failed**. 
- Note that you will first need to aggregate the grades table by the course uuid to include all sections. Show the name of the course as well that you will need to get from the course_offerings table.

In [14]:
step1 = course_offerings.join(grade_distributions,course_offerings.uuid==grade_distributions.course_offering_uuid)\
        .select([course_offerings.name.alias("course_name")] +grade_distributions.columns)


In [15]:
step1.columns

['course_name',
 'course_offering_uuid',
 'section_number',
 'a_count',
 'ab_count',
 'b_count',
 'bc_count',
 'c_count',
 'd_count',
 'f_count',
 's_count',
 'u_count',
 'cr_count',
 'n_count',
 'p_count',
 'i_count',
 'nw_count',
 'nr_count',
 'other_count']

In [16]:
from pyspark.sql.functions import *
#step1.groupBy("course_name").agg(sum("num_of_f").alias("failure"), agg).orderBy(desc("sum(num_of_f)")).toPandas()

step2 = step1.drop('course_offering_uuid', 'section_number', 's_count',
 'u_count',
 'cr_count',
 'n_count',
 'p_count',
 'i_count',
 'nw_count',
 'nr_count',
 'other_count').groupBy("course_name").sum()

step2.toPandas()

,course_name,sum(a_count),sum(ab_count),sum(b_count),sum(bc_count),sum(c_count),sum(d_count),sum(f_count)
0,Math for Tchng:,167,105,155,60,56,21,8
1,Where Science Meets Society,140,157,73,8,11,3,2
2,19&20 C Russian Lit Tran I,440,179,74,23,17,8,10
3,Elem/Intmed Trumpet,77,38,6,1,0,0,0
4,Elementary Naval Lab I,156,6,0,1,0,0,1
...,...,...,...,...,...,...,...,...
8238,Teaching Science-Sec Schl,89,20,6,0,0,0,0
8239,Environmental Governance,7,0,1,0,0,0,0
8240,Media History & Historiography,11,1,0,0,0,0,0
8241,Occupational Audiology,17,1,0,0,0,0,0


In [17]:
step2.columns

['course_name',
 'sum(a_count)',
 'sum(ab_count)',
 'sum(b_count)',
 'sum(bc_count)',
 'sum(c_count)',
 'sum(d_count)',
 'sum(f_count)']

In [18]:
step3 = step2.withColumn("Non-F-Count",step2["sum(a_count)"]+step2["sum(ab_count)"]+step2["sum(b_count)"]+step2["sum(bc_count)"]+step2["sum(c_count)"]+step2["sum(d_count)"])\
        .drop( 'sum(a_count)', 'sum(ab_count)', 'sum(b_count)', 'sum(bc_count)', 'sum(c_count)', 'sum(d_count)').withColumnRenamed("sum(f_count)","f_count")
step3.toPandas()

,course_name,f_count,Non-F-Count
0,Math for Tchng:,8,564
1,Where Science Meets Society,2,392
2,19&20 C Russian Lit Tran I,10,741
3,Elem/Intmed Trumpet,0,122
4,Elementary Naval Lab I,1,163
...,...,...,...
8238,Teaching Science-Sec Schl,0,115
8239,Environmental Governance,0,8
8240,Media History & Historiography,0,12
8241,Occupational Audiology,0,18


In [19]:
step3.select(step3.course_name,(step3.f_count/step3["Non-F-Count"]).alias("F_rate")).orderBy(desc("F_rate")).toPandas()

,course_name,F_rate
0,Advanced Linear Programming,0.333333
1,Afr Am Women's Activism 19-20C,0.285714
2,Adv Intdis Stdy-Medvl Civ,0.200000
3,Field Methods-Geol Engr,0.200000
4,Fourth Semester Filipino,0.200000
...,...,...
8238,"5th Sem, Lang of W Africa",NaN
8239,Preceptorship-Ashland,NaN
8240,Adt Orth-Non-Orth Clerk-CSC,NaN
8241,PhD Diss/DMA Project,NaN


## Challenge Question: Automating data entry errors

We see in the dataframe below that there are several typos of various animal names. If this was a large database of several millions of records, correcting these errors would be way too labor intensive. How can we automate correcting these errors?

*Hint: Leven...*

In [20]:
values = [('Monkey',10),('Monkay',36),('Mnky',123), \
          ('Elephant',48),('Elefant',16),('Ellafant',1), \
          ('Hippopotamus',48),('Hipopotamus',16),('Hippo',1)]

zoo = spark.createDataFrame(values,['Animal','age'])
zoo.show()

+------------+---+
|      Animal|age|
+------------+---+
|      Monkey| 10|
|      Monkay| 36|
|        Mnky|123|
|    Elephant| 48|
|     Elefant| 16|
|    Ellafant|  1|
|Hippopotamus| 48|
| Hipopotamus| 16|
|       Hippo|  1|
+------------+---+



In [21]:
#with the levenshtein distance!
from pyspark.sql.functions import levenshtein
from pyspark.sql.types import *

#Firstly, create dataframe with the 3 options we want to choose from
options = spark.createDataFrame(["Monkey","Elephant","Hippopotamus"],StringType())
options.show()

+------------+
|       value|
+------------+
|      Monkey|
|    Elephant|
|Hippopotamus|
+------------+



In [22]:
#And then we joint the two Dataframes together with a condition >5
results = zoo.join(options, levenshtein(zoo["Animal"],options["value"])<5,how="left")
results.select("Animal","age","value",levenshtein(zoo["Animal"],options["value"])).show()

+------------+---+------------+--------------------------+
|      Animal|age|       value|levenshtein(Animal, value)|
+------------+---+------------+--------------------------+
|      Monkey| 10|      Monkey|                         0|
|      Monkay| 36|      Monkey|                         1|
|        Mnky|123|      Monkey|                         2|
|    Elephant| 48|    Elephant|                         0|
|     Elefant| 16|    Elephant|                         2|
|    Ellafant|  1|    Elephant|                         3|
|Hippopotamus| 48|Hippopotamus|                         0|
| Hipopotamus| 16|Hippopotamus|                         1|
|       Hippo|  1|        null|                      null|
+------------+---+------------+--------------------------+



### Great job! 